#1 - Importando Bibliotecas

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, current_date
import os
from pyspark.sql.types import IntegerType, DoubleType, FloatType, StringType
from pyspark.sql.functions import col, lit, when

#2 - Exibindo ponto de montagem 

In [0]:
display(dbutils.fs.ls("/mnt/data"))

path,name,size,modificationTime
dbfs:/mnt/data/processed/,processed/,0,0
dbfs:/mnt/data/raw/,raw/,0,0
dbfs:/mnt/data/refined/,refined/,0,0


#3 - Leitura dos dados na camada Raw

In [0]:
# Listar os arquivos na pasta 'tabelas'
files = dbutils.fs.ls("/mnt/data/raw/tabelas")

# Iterar sobre cada arquivo e exibir seu conteúdo
for file in files:
    print(f"Lendo o arquivo: {file.name}")  # Exibir o nome do arquivo
    df_csv = spark.read.format("csv").option("header", "true").option("sep", ";").option("inferSchema", "true").load(file.path)  # Carregar o arquivo CSV atual
    
    # Exibir o conteúdo do DataFrame
#    display(df_csv)


Lendo o arquivo: categories.csv
Lendo o arquivo: customer_customer_demo.csv
Lendo o arquivo: customer_demographics.csv
Lendo o arquivo: customers.csv
Lendo o arquivo: employee_territories.csv
Lendo o arquivo: employees.csv
Lendo o arquivo: order_details.csv
Lendo o arquivo: orders.csv
Lendo o arquivo: products.csv
Lendo o arquivo: region.csv
Lendo o arquivo: shippers.csv
Lendo o arquivo: suppliers.csv
Lendo o arquivo: territories.csv
Lendo o arquivo: us_states.csv


#4 - Transformando os arquivos em parquet e salvando na camada Processed

In [0]:
# Caminhos de entrada e saída
input_directory = "/mnt/data/raw/tabelas"
output_directory = "/mnt/data/processed/tabelas_parquet/"

# Listar todos os arquivos na pasta raw/tabelas
files = dbutils.fs.ls(input_directory)

# Processar cada arquivo CSV
for file in files:
    file_name = file.name  # Nome do arquivo
    if file_name.endswith(".csv"):  # Garantir que o arquivo seja CSV
        print(f"Processando arquivo: {file_name}")
        
        # Caminho completo do arquivo na pasta raw
        input_path = f"{input_directory}/{file_name}"
        
        # Nome do arquivo Parquet de saída (adaptado ao nome original do CSV)
        output_file = f"{output_directory}rw_northwind_{file_name.replace('.csv', '.parquet')}"
        
        # Ler o arquivo CSV
        df = spark.read.format("csv").option("header", "true").option("sep", ";").option("inferSchema", "true").load(input_path)
        
        # Reduzir para uma única partição
        df_single_partition = df.coalesce(1)
        
        # Salvar em um diretório temporário
        temp_output_path = f"{output_directory}temp_output_{file_name.replace('.csv', '')}"
        df_single_partition.write.mode("overwrite").parquet(temp_output_path)
        
        # Identificar o arquivo Parquet gerado
        temp_files = dbutils.fs.ls(temp_output_path)
        parquet_file = [file.path for file in temp_files if file.path.endswith(".parquet")][0]
        
        # Renomear o arquivo Parquet para o nome final
        dbutils.fs.mv(parquet_file, output_file)
        
        # Remover o diretório temporário
        dbutils.fs.rm(temp_output_path, recurse=True)
        
        print(f"Arquivo {file_name} salvo como: {output_file}")

print("Processamento concluído para todos os arquivos.")

Processando arquivo: categories.csv
Arquivo categories.csv salvo como: /mnt/data/processed/tabelas_parquet/rw_northwind_categories.parquet
Processando arquivo: customer_customer_demo.csv
Arquivo customer_customer_demo.csv salvo como: /mnt/data/processed/tabelas_parquet/rw_northwind_customer_customer_demo.parquet
Processando arquivo: customer_demographics.csv
Arquivo customer_demographics.csv salvo como: /mnt/data/processed/tabelas_parquet/rw_northwind_customer_demographics.parquet
Processando arquivo: customers.csv
Arquivo customers.csv salvo como: /mnt/data/processed/tabelas_parquet/rw_northwind_customers.parquet
Processando arquivo: employee_territories.csv
Arquivo employee_territories.csv salvo como: /mnt/data/processed/tabelas_parquet/rw_northwind_employee_territories.parquet
Processando arquivo: employees.csv
Arquivo employees.csv salvo como: /mnt/data/processed/tabelas_parquet/rw_northwind_employees.parquet
Processando arquivo: order_details.csv
Arquivo order_details.csv salvo co

In [0]:
# Lista de arquivos CSV originais processados
csv_files = [
    "customers",
    "orders",
    "products",
    "categories",
    "suppliers",
    "order_details",
    "employees",
    "territories",
    "region",
    "shippers",
    "customer_customer_demo",
    "customer_demographics",
    "employee_territories",
    "us_states"
]

# Loop para processar cada arquivo
for file_name in csv_files:
    print(f"\n\n----- Processando o arquivo: {file_name} -----")
    
    # Caminho do arquivo Parquet na camada processed
    parquet_file = f"/mnt/data/processed/tabelas_parquet/rw_northwind_{file_name}.parquet"
    
    try:
        # Ler o arquivo Parquet
        df_parquet = spark.read.format("parquet").load(parquet_file)
        
        # Exibir o esquema da tabela
        print(f"Esquema do DataFrame para {file_name}:")
        df_parquet.printSchema()
        
        # Exibir os dados de forma tabulada
        print(f"Visualizando os dados de {file_name}:")
        display(df_parquet)
    
    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")




----- Processando o arquivo: customers -----
Esquema do DataFrame para customers:
root
 |-- customer_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- contact_name: string (nullable = true)
 |-- contact_title: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- fax: string (nullable = true)

Visualizando os dados de customers:


customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,null,12209,Germany,030-0074321,030-0076545
ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,null,05021,Mexico,(5) 555-4729,(5) 555-3745
ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,null,05023,Mexico,(5) 555-3932,null
AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,null,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,null,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
BLAUS,Blauer See Delikatessen,Hanna Moos,Sales Representative,Forsterstr. 57,Mannheim,null,68306,Germany,0621-08460,0621-08924
BLONP,Blondesddsl père et fils,Frédérique Citeaux,Marketing Manager,"24, place Kléber",Strasbourg,null,67000,France,88.60.15.31,88.60.15.32
BOLID,Bólido Comidas preparadas,Martín Sommer,Owner,"C/ Araquil, 67",Madrid,null,28023,Spain,(91) 555 22 82,(91) 555 91 99
BONAP,Bon app',Laurence Lebihan,Owner,"12, rue des Bouchers",Marseille,null,13008,France,91.24.45.40,91.24.45.41
BOTTM,Bottom-Dollar Markets,Elizabeth Lincoln,Accounting Manager,23 Tsawassen Blvd.,Tsawassen,BC,T2F 8M4,Canada,(604) 555-4729,(604) 555-3745




----- Processando o arquivo: orders -----
Esquema do DataFrame para orders:
root
 |-- order_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- required_date: date (nullable = true)
 |-- shipped_date: date (nullable = true)
 |-- ship_via: integer (nullable = true)
 |-- freight: double (nullable = true)
 |-- ship_name: string (nullable = true)
 |-- ship_address: string (nullable = true)
 |-- ship_city: string (nullable = true)
 |-- ship_region: string (nullable = true)
 |-- ship_postal_code: string (nullable = true)
 |-- ship_country: string (nullable = true)

Visualizando os dados de orders:


order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,null,51100,France
10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,null,44087,Germany
10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,null,69004,France
10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.3,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,null,B-6000,Belgium
10253,HANAR,3,1996-07-10,1996-07-24,1996-07-16,2,58.17,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
10254,CHOPS,5,1996-07-11,1996-08-08,1996-07-23,2,22.98,Chop-suey Chinese,Hauptstr. 31,Bern,null,3012,Switzerland
10255,RICSU,9,1996-07-12,1996-08-09,1996-07-15,3,148.33,Richter Supermarkt,Starenweg 5,Genève,null,1204,Switzerland
10256,WELLI,3,1996-07-15,1996-08-12,1996-07-17,2,13.97,Wellington Importadora,"Rua do Mercado, 12",Resende,SP,08737-363,Brazil
10257,HILAA,4,1996-07-16,1996-08-13,1996-07-22,3,81.91,HILARION-Abastos,Carrera 22 con Ave. Carlos Soublette #8-35,San Cristóbal,Táchira,5022,Venezuela




----- Processando o arquivo: products -----
Esquema do DataFrame para products:
root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- supplier_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- quantity_per_unit: string (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- units_in_stock: integer (nullable = true)
 |-- units_on_order: integer (nullable = true)
 |-- reorder_level: integer (nullable = true)
 |-- discontinued: integer (nullable = true)

Visualizando os dados de products:


...

customer_type_id,customer_desc




----- Processando o arquivo: employee_territories -----
Esquema do DataFrame para employee_territories:
root
 |-- employee_id: integer (nullable = true)
 |-- territory_id: integer (nullable = true)

Visualizando os dados de employee_territories:


employee_id,territory_id
1,6897
1,19713
2,1581
2,1730
2,1833
2,2116
2,2139
2,2184
2,40222
3,30346




----- Processando o arquivo: us_states -----
Esquema do DataFrame para us_states:
root
 |-- state_id: integer (nullable = true)
 |-- state_name: string (nullable = true)
 |-- state_abbr: string (nullable = true)
 |-- state_region: string (nullable = true)

Visualizando os dados de us_states:


state_id,state_name,state_abbr,state_region
1,Alabama,AL,south
2,Alaska,AK,north
3,Arizona,AZ,west
4,Arkansas,AR,south
5,California,CA,west
6,Colorado,CO,west
7,Connecticut,CT,east
8,Delaware,DE,east
9,District of Columbia,DC,east
10,Florida,FL,south


#5 - Validando Dados

In [0]:
from pyspark.sql.types import StructType

# Lista de arquivos na processed
processed_files = [
    "customers", "orders", "products", "categories", "suppliers",
    "order_details", "employees", "territories", "region",
    "shippers", "customer_customer_demo", "customer_demographics",
    "employee_territories", "us_states"
]

# Validação dos esquemas
for file_name in processed_files:
    print(f"\nValidando o esquema para: {file_name}")
    
    # Caminho do arquivo Parquet
    parquet_file = f"/mnt/data/processed/tabelas_parquet/rw_northwind_{file_name}.parquet"
    
    try:
        # Ler o DataFrame
        df = spark.read.format("parquet").load(parquet_file)
        
        # Exibir o esquema
        print(f"Esquema de {file_name}:")
        df.printSchema()
    except Exception as e:
        print(f"Erro ao validar o esquema de {file_name}: {e}")



Validando o esquema para: customers
Esquema de customers:
root
 |-- customer_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- contact_name: string (nullable = true)
 |-- contact_title: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- fax: string (nullable = true)


Validando o esquema para: orders
Esquema de orders:
root
 |-- order_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- required_date: date (nullable = true)
 |-- shipped_date: date (nullable = true)
 |-- ship_via: integer (nullable = true)
 |-- freight: double (nullable = true)
 |-- ship_name: string (nullable = true)
 |-- ship_address: string (nullable = true)
 |-- ship_city: 

In [0]:
# Lista de arquivos na processed
processed_files = [
    "customers", "orders", "products", "categories", "suppliers",
    "order_details", "employees", "territories", "region",
    "shippers", "customer_customer_demo", "customer_demographics",
    "employee_territories", "us_states"
]

# Lista para armazenar resultados
resultados_validacao = []

# Loop para processar cada tabela
for file_name in processed_files:
    parquet_file = f"/mnt/data/processed/tabelas_parquet/rw_northwind_{file_name}.parquet"
    print(f"\nValidando: {file_name}")
    
    try:
        # Ler o DataFrame
        df = spark.read.format("parquet").load(parquet_file)
        
        # Verificar nulos
        nulos = {col: df.filter(df[col].isNull()).count() for col in df.columns}
        total_nulos = sum(nulos.values())
        
        # Verificar duplicatas
        duplicatas = df.count() - df.dropDuplicates().count()
        
        # Adicionar resultado
        status = "OK" if total_nulos == 0 and duplicatas == 0 else "Erro"
        resultados_validacao.append((file_name, status, total_nulos, duplicatas))
    
    except Exception as e:
        print(f"Erro ao processar {file_name}: {e}")
        resultados_validacao.append((file_name, "Erro", None, None))

# Criar DataFrame com os resultados
df_resultados = spark.createDataFrame(
    resultados_validacao,
    ["Tabela", "Status", "Nulos", "Duplicatas"]
)

# Salvar resultados em Parquet
output_path = "/mnt/data/processed/validacao/relatorio_validacao.parquet"
df_resultados.write.mode("overwrite").parquet(output_path)

# Exibir resultados
display(df_resultados)



Validando: customers

Validando: orders

Validando: products

Validando: categories

Validando: suppliers

Validando: order_details

Validando: employees

Validando: territories

Validando: region

Validando: shippers

Validando: customer_customer_demo

Validando: customer_demographics

Validando: employee_territories

Validando: us_states


Tabela,Status,Nulos,Duplicatas
customers,Erro,83,0
orders,Erro,547,0
products,OK,0,0
categories,OK,0,0
suppliers,Erro,60,0
order_details,OK,0,0
employees,Erro,5,0
territories,OK,0,0
region,OK,0,0
shippers,OK,0,0


In [0]:
def treat_nulls_by_type_and_save(table_name, table_path, output_path):
    print(f"Tratando valores nulos para a tabela: {table_name}")

    # Ler a tabela Parquet
    df = spark.read.format("parquet").load(table_path)

    # Tratar nulos com base no tipo de dado
    for column, dtype in df.dtypes:
        if dtype in ["int", "bigint", "float", "double"]:  # Colunas numéricas
            df = df.withColumn(column, when(col(column).isNull(), lit(0)).otherwise(col(column)))
        elif dtype == "string":  # Colunas string
            df = df.withColumn(column, when(col(column).isNull(), lit("")).otherwise(col(column)))
        elif dtype == "date":  # Colunas de data
            df = df.withColumn(column, when(col(column).isNull(), lit("1900-01-01")).otherwise(col(column)))

    # Salvar a tabela tratada na camada processed
    output_file = f"{output_path}/{table_name}_treated.parquet"
    df.write.mode("overwrite").parquet(output_file)
    print(f"Tabela '{table_name}' tratada e salva em: {output_file}")

# Caminho da camada processed
processed_path = "/mnt/data/processed/treated_files"

# Tabelas com valores nulos identificadas
tables_with_nulls = [
    ("customers", "/mnt/data/processed/tabelas_parquet/rw_northwind_customers.parquet"),
    ("orders", "/mnt/data/processed/tabelas_parquet/rw_northwind_orders.parquet"),
    ("suppliers", "/mnt/data/processed/tabelas_parquet/rw_northwind_suppliers.parquet"),
    ("employees", "/mnt/data/processed/tabelas_parquet/rw_northwind_employees.parquet")
]

# Iterar pelas tabelas e tratar nulos
for table_name, table_path in tables_with_nulls:
    treat_nulls_by_type_and_save(table_name, table_path, processed_path)


Tratando valores nulos para a tabela: customers
Tabela 'customers' tratada e salva em: /mnt/data/processed/treated_files/customers_treated.parquet
Tratando valores nulos para a tabela: orders
Tabela 'orders' tratada e salva em: /mnt/data/processed/treated_files/orders_treated.parquet
Tratando valores nulos para a tabela: suppliers
Tabela 'suppliers' tratada e salva em: /mnt/data/processed/treated_files/suppliers_treated.parquet
Tratando valores nulos para a tabela: employees
Tabela 'employees' tratada e salva em: /mnt/data/processed/treated_files/employees_treated.parquet


In [0]:
from pyspark.sql.functions import col, lit, when

# Caminho corrigido das tabelas tratadas
treated_tables = {
    "customers": "/mnt/data/processed/treated_files/customers_treated.parquet",
    "orders": "/mnt/data/processed/treated_files/orders_treated.parquet",
    "suppliers": "/mnt/data/processed/treated_files/suppliers_treated.parquet",
    "employees": "/mnt/data/processed/treated_files/employees_treated.parquet",
}

# Caminho para salvar os arquivos finais
final_output_path = "/mnt/data/processed/final_files"

# Iterar pelas tabelas tratadas
for table_name, table_path in treated_tables.items():
    try:
        print(f"Lendo e salvando tabela: {table_name}")
        
        # Ler o arquivo tratado
        df = spark.read.format("parquet").load(table_path)
        
        # Unir todas as partições em um único arquivo
        df = df.coalesce(1)
        
        # Salvar em uma pasta temporária
        temp_output_path = f"{final_output_path}/temp_{table_name}"
        df.write.mode("overwrite").parquet(temp_output_path)
        
        # Identificar e mover o arquivo Parquet único
        temp_dir = f"/dbfs{temp_output_path}"
        final_file_name = f"{table_name}.parquet"
        final_file_path = f"/dbfs{final_output_path}/{final_file_name}"
        
        # Encontrar o arquivo gerado automaticamente
        for file in os.listdir(temp_dir):
            if file.endswith(".parquet"):
                shutil.move(f"{temp_dir}/{file}", final_file_path)
                break
        
        # Remover a pasta temporária
        shutil.rmtree(temp_dir)
        
        print(f"Tabela {table_name} salva como arquivo único em: {final_file_path}")
    
    except Exception as e:
        print(f"Erro ao processar a tabela {table_name}: {e}")


Lendo e salvando tabela: customers
Erro ao processar a tabela customers: [Errno 22] Invalid argument: '/dbfs/mnt/data/processed/final_files/temp_customers'
Lendo e salvando tabela: orders
Erro ao processar a tabela orders: [Errno 22] Invalid argument: '/dbfs/mnt/data/processed/final_files/temp_orders'
Lendo e salvando tabela: suppliers
Erro ao processar a tabela suppliers: [Errno 22] Invalid argument: '/dbfs/mnt/data/processed/final_files/temp_suppliers'
Lendo e salvando tabela: employees
Erro ao processar a tabela employees: [Errno 22] Invalid argument: '/dbfs/mnt/data/processed/final_files/temp_employees'


In [0]:
import os
from pyspark.dbutils import DBUtils

dbutils = DBUtils(spark)

# Caminho do diretório final
final_output_path = "/mnt/data/processed/final_files"

# Mapear os nomes antigos para os novos
file_renames = {
    "rw_customers_final.parquet": "rw_northwind_customers_final.parquet",
    "rw_employees_final.parquet": "rw_northwind_employees_final.parquet",
    "rw_orders_final.parquet": "rw_northwind_orders_final.parquet",
    "rw_suppliers_final.parquet": "rw_northwind_suppliers_final.parquet",
}

# Iterar pelos arquivos para renomear
for old_name, new_name in file_renames.items():
    old_path = f"{final_output_path}/{old_name}"
    new_path = f"{final_output_path}/{new_name}"
    try:
        # Mover (renomear) o arquivo
        dbutils.fs.mv(old_path, new_path)
        print(f"Arquivo {old_name} renomeado para {new_name}")
    except Exception as e:
        print(f"Erro ao renomear {old_name} para {new_name}: {e}")

Erro ao renomear rw_customers_final.parquet para rw_northwind_customers_final.parquet: An error occurred while calling o414.mv.
: java.io.FileNotFoundException: /mnt/data/processed/final_files/rw_customers_final.parquet
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$2(DatabricksFileSystemV2.scala:1199)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.$anonfun$getFileStatus$1(DatabricksFileSystemV2.scala:1185)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:527)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:631)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:651)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.sca

In [0]:
from pyspark.dbutils import DBUtils

dbutils = DBUtils(spark)

# Caminhos das pastas
source_path = "/mnt/data/processed/tabelas_parquet"
target_path = "/mnt/data/processed/final_files"

# Mapear os arquivos a serem movidos
files_to_move = [
    "rw_northwind_categories.parquet",
    "rw_northwind_customer_customer_demo.parquet",
    "rw_northwind_customer_demographics.parquet",
    "rw_northwind_employee_territories.parquet",
    "rw_northwind_order_details.parquet",
    "rw_northwind_products.parquet",
    "rw_northwind_region.parquet",
    "rw_northwind_shippers.parquet",
    "rw_northwind_territories.parquet",
    "rw_northwind_us_states.parquet"
]

# Mover e renomear arquivos
for file_name in files_to_move:
    old_path = f"{source_path}/{file_name}"
    new_name = file_name.replace(".parquet", "_final.parquet")
    new_path = f"{target_path}/{new_name}"
    
    try:
        dbutils.fs.mv(old_path, new_path)
        print(f"Arquivo {file_name} movido para {new_path}")
    except Exception as e:
        print(f"Erro ao mover o arquivo {file_name}: {e}")


Arquivo rw_northwind_categories.parquet movido para /mnt/data/processed/final_files/rw_northwind_categories_final.parquet
Arquivo rw_northwind_customer_customer_demo.parquet movido para /mnt/data/processed/final_files/rw_northwind_customer_customer_demo_final.parquet
Arquivo rw_northwind_customer_demographics.parquet movido para /mnt/data/processed/final_files/rw_northwind_customer_demographics_final.parquet
Arquivo rw_northwind_employee_territories.parquet movido para /mnt/data/processed/final_files/rw_northwind_employee_territories_final.parquet
Arquivo rw_northwind_order_details.parquet movido para /mnt/data/processed/final_files/rw_northwind_order_details_final.parquet
Arquivo rw_northwind_products.parquet movido para /mnt/data/processed/final_files/rw_northwind_products_final.parquet
Arquivo rw_northwind_region.parquet movido para /mnt/data/processed/final_files/rw_northwind_region_final.parquet
Arquivo rw_northwind_shippers.parquet movido para /mnt/data/processed/final_files/rw_n